In [449]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
# 데이터 읽어서 DataFrame에 저장
train_df = pd.read_csv("ratings_train.txt", "\t")
test_df = pd.read_csv("ratings_test.txt", "\t")

In [450]:
train_df = train_df[:1500]
test_df = test_df[:1000]
# test_df = test_df.dropna(axis = 0)
# train_df = train_df.dropna(axis = 0)

In [451]:
isPositive = train_df["label"] == 1
isNegative = train_df["label"] == 0
train_df_P = train_df[isPositive]
train_df_N = train_df[isNegative]

In [452]:
docP, docN = "", ""
for s in train_df_P["document"]:
    docP += str(s)
for s in train_df_N["document"]:
    docN += str(s)

In [453]:
from konlpy.tag import Twitter
class MyTokenizer:
    def __init__(self, tagger):
        self.tagger = tagger
    def __call__(self, sent):
        pos = self.tagger.pos(sent)
        ret = []
#         pos = ['{}/{}'.format(word,tag) for word, tag in pos]
        for word, tag in pos:
            if not(tag == 'Josa' or tag == 'Punctuation' or tag == 'Number' or tag == 'Modifier' or tag == 'Eomi'):
                ret.append(word)
        return ret
my_tokenizer = MyTokenizer(Twitter())

/home/inns/.local/lib/python3.7/site-packages/konlpy/tag/_okt.py:16: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


In [454]:
doc = [docP, docN]

In [455]:
tfidf = TfidfVectorizer(tokenizer = my_tokenizer, max_features = 2000, max_df = 0.97, min_df = 0.03)
X = tfidf.fit_transform(doc)
tfidf_dict = tfidf.get_feature_names()

In [456]:
# 단어사전의 단어를 포함하고 있지 않으면 없애벌임
def isTfidf(document):
    for word in tfidf_dict:
        if(word in document):
            return True
    return False

In [457]:
is_tfidf = pd.DataFrame()
for idx in range(len(train_df)):
    if isTfidf(train_df["document"][idx]) == True:
        is_tfidf = is_tfidf.append(train_df.loc[idx])

In [458]:
vectorizer = CountVectorizer(tokenizer = my_tokenizer)
feature_train_X = vectorizer.fit_transform(is_tfidf["document"])

In [459]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
classifier = MultinomialNB()
classifier.fit(feature_train_X, is_tfidf["label"])
feature_test_X = vectorizer.transform(test_df["document"])
predictions = classifier.predict(feature_test_X).tolist()
print('Accuracy: %.10f' % accuracy_score(test_df["label"], predictions))

Accuracy: 0.7610000000


In [460]:
def cut_data():
    data_ham, data_spam = [], []
    for i in range(len(x_data)):
        if(y_data[i] == 'ham'):
            data_ham.append(x_data[i])
        else:
            data_spam.append(x_data[i])
    return data_ham, data_spam
def wordCount(data, w):
    cnt = 0;
    for s in data:
        if w in s:
            cnt += 1
    return cnt;
def getKai(data, w, c):
    A = wordCount(data[c], w)
    B = wordCount(data[(c+1)%2], w)
    C = len(data[c]) - A
    D = len(data[(c+1)%2]) - B
    if ((A+B)*(A+C)*(B+D)*(C+D)) == 0:
        return 0
    return ((A+B+C+D)*((A*D-B*C)*(A*D-B*C))) / ((A+B)*(A+C)*(B+D)*(C+D))

In [461]:
len(is_tfidf)

1414

In [462]:
is_tfidf_P_bool = is_tfidf["label"] == 1
is_tfidf_N_bool = is_tfidf["label"] == 0

In [463]:
is_tfidf_P = is_tfidf[is_tfidf_P_bool]
is_tfidf_N = is_tfidf[is_tfidf_N_bool]

In [464]:
data = []
data.append(list(is_tfidf_N["document"]))
data.append(list(is_tfidf_P["document"]))

In [465]:
words_spam, words_spam_kai= [], []
words_spam_set = set(words_spam)
words_spam = list(words_spam_set)

data = []
data.append(list(train_df_N["document"]))
data.append(list(train_df_P["document"]))
            
for w in tfidf_dict:
    words_spam_kai.append(getKai(data, w, 0))

kaiRate = pd.DataFrame()
kaiRate['words'] = tfidf_dict
kaiRate.index = tfidf_dict
kaiRate['word_spam_kai'] = words_spam_kai

kaiRate = kaiRate.sort_values(by=['word_spam_kai'],ascending=False)

In [466]:
kaiRate = kaiRate[:1000]

In [467]:
# kai 단어사전의 단어를 포함하고 있지 않으면 없애벌임
def isKai(document):
    for word in kaiRate["words"]:
        if(word in str(document)):
            return True
    return False

In [468]:
is_Kai = pd.DataFrame()
for idx in range(len(train_df)):
    if isKai(train_df["document"][idx]) == True:
        is_Kai = is_Kai.append(train_df.loc[idx])
        

In [469]:
len(is_Kai)

1347

In [470]:
vectorizer_kai = CountVectorizer(tokenizer = my_tokenizer)
feature_train_X_kai = vectorizer.fit_transform(is_Kai["document"])

In [471]:
# 합칠 때 오류 나옮

In [472]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
classifier = MultinomialNB()
classifier.fit(feature_train_X_kai, is_Kai["label"])
feature_test_X = vectorizer.transform(test_df["document"])
predictions_ = classifier.predict(feature_test_X).tolist()
print('Accuracy: %.10f' % accuracy_score(test_df["label"], predictions))

Accuracy: 0.7610000000


In [479]:
# 애매한 문장 찾기
for t in range(500):
    if(float(predictions_[t]) !=  float(predictions[t])):
        print(str(t) + test_df["document"][t])

31안보면 후회ㅠㅠ...
47마음이1을보신분들은 이해하시겟지만 2는망장입니다 졸작이죠 솔직히말해선;; 그이유는 예를들면 엽기적인그녀는 긴시간을두고 코메디 멜로를 그렷습니다 하지만 이영화는 단시간 10분안에 모든것을 소화하려는 욕심 코믹하다 3분채안돼갑자기 생뚱맞게 슬픈음악이깔리고
56역사얘기보다 영화배우, 감독들들에게 더 관심이 간다
117피는 물보다 진하다. 이 한마디로 모든게 표현되는 영화.
272이젠 양동근은 믿고보는 배우가 아니게 되어버렸다.
275나는 다르게 생각하는데... 여기서 너구리에서 인간으로 변신해서 살아가는 사람들은 우리 사회의 소수자들을 상징하는 것 같다.
329후..
39530대 여자의 노처녀 히스테리에 열등감이 더해지면 어떻게 되는지를 보여주는 영화. 골때리고 끝내주게 웃겨준다
439자극적이기 위해 끼운 억지스런 장면들. 속보이는 뻔한 계산.연출성은 그렇다치고 감독의 인격수준이 바닥.


In [474]:
predictions

[1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.0,
 0.0,
 1.0,
 1.0,
 0.0,
 1.0,
 1.0,
 1.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 1.0,
 0.0,
 1.0,
 0.0,
 1.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 1.0,
 1.0,
 1.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 1.0,
 0.0,
 1.0,
 1.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 1.0,
 1.0,
 0.0,
 0.0,
 1.0,
 0.0,
 1.0,
 0.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 1.0,
 1.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 1.0,
 1.0,
 0.0,
 1.0,
 0.0,
 1.0,
 0.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.0,
 0.0,
 1.0,
 1.0,
 0.0,
 0.0,
 0.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.0,
 1.0,
 0.0,
 1.0,
 1.0,
 0.0,
 1.0,
 0.0,
 1.0,
 0.0,
 1.0,
 0.0,
 1.0,
 0.0,
 0.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.0,
 0.0,
 1.0,
 1.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 1.0,
 1.0,
 0.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.0,
 1.0,
 0.0,
 1.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0

In [475]:
predictions_

[1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.0,
 0.0,
 1.0,
 1.0,
 0.0,
 1.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 1.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 1.0,
 0.0,
 1.0,
 1.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 1.0,
 1.0,
 0.0,
 0.0,
 1.0,
 0.0,
 1.0,
 0.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 1.0,
 1.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 1.0,
 1.0,
 0.0,
 1.0,
 0.0,
 1.0,
 0.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.0,
 0.0,
 1.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 1.0,
 1.0,
 0.0,
 1.0,
 0.0,
 1.0,
 1.0,
 0.0,
 1.0,
 0.0,
 1.0,
 0.0,
 1.0,
 0.0,
 1.0,
 0.0,
 0.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.0,
 0.0,
 1.0,
 1.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 1.0,
 1.0,
 0.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.0,
 1.0,
 0.0,
 1.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0